# Discraft

Discraft is a powerful framework for creating Discord bots, offering a robust CLI and a set of tools to streamline the development process.
Think of it like Next.js but for Discord bots.

## Layout of Project

The source code for Discord bots goes in the `src/` directory.

### Commands

`src/commands/` contains command files for the bot.
It's considered a good practice to name each file the same as the desired `/` (application) command name.
By default, there is one command on initialization, the ping command.
Looking at [ping.js](src/commands/ping.js), you should notice that `SlashCommandBuilder` is imported from `discord.js`. This is used to build the command, including metadata like name and description.
The `execute` function is where the command's logic goes. It is executed when the command is called.

**Rules**

- Command files must end in with the `.js` extension.
- The command must export both an `execute` function and a `data` object.
- The command file name cannot contain numbers.

**Best Practices**

- Use `SlashCommandBuilder` to create the data object.
- The `execute` function should be asynchronous.

**Example**
See below:

```javascript
import { SlashCommandBuilder } from "discord.js";

export default {
  data: new SlashCommandBuilder()
    .setName("ping")
    .setDescription("Replies with Pong"),

  async execute(interaction) {
    const sent = await interaction.reply({ content: "Pinging...", fetchReply: true });
    const latency = sent.createdTimestamp - interaction.createdTimestamp;
    await interaction.editReply(`Pong!\nLatency: ${latency}ms\nWS Ping: ${Math.round(interaction.client.ws.ping)}ms`);
  },
};
```

### Events

`src/events/` contains event files for the bot.
By default, there are two events on initialization, the ready event and the error event.

Let's take a look at [ready.js](src/events/ready.js):

```javascript
import { debug, error } from "../utils/logger.js";
import { ActivityType, Events } from "discord.js";

export default (client) => {
    client.on(Events.ClientReady, () => {
        debug("Setting presence...");
        try {
            client.user.setPresence({
                activities: [{ name: "Discraft", emoji: "", state: "Created with Discraft", type: ActivityType.Custom }],
                status: "online"
            });
        } catch (err) {
            error("Error setting presence:", err);
        }
    });
};
```

As you can see, we import the `debug` and `error` functions from the `logger.js` file.
This is used to log messages and errors in the console.
We also import the `ActivityType` and `Events` constants from the `discord.js` module.
These are used to set the bot's presence. You can import whatever you need in custom event files.

**Rules**

- Event files must end in with the `.js` extension.
- The event must export a default export, like this: `export default (client) => {...}`.
- The event file name cannot contain numbers.

**Best Practices**

- Implement good error handling in the event file.
- Error handling should be contained inside the default export function.

### Utilities

`src/utils/` contains utility files for the bot.
By default, utils contains just the logger.js file, which exports console functions like `log`, `error`, `debug`, etc. with enhanced formatting.
You create any custom utility you want to use in other files in the project. They aren't dynamically loaded or generated on build, so they don't have too many specific rules.

**Best Practices**

- Implement good error handling in the utility file.
- The files should probably end with `.js`; avoid using `.json` files.

### Configuration

`src/config/` contains configuration files for the bot. By default, there is just the `bot.config.js` file, which exports ENV variables as constants and can use JavaScript to compute environment values, unlike the `.env` file.
You shouldn't mess with the bot config file too much. You can add new exports to it if you want, but it's not recommended to change the existing ones.
You can also create new config files in the `src/config/` directory.

**Best Practices**

- Use the `bot.config.js` instead of `process.env` when you write more scripts.
- Do not create `.env` files in the `src/config/` directory, it's not the proper directory and most `.gitignore` files won't omit them by default. You don't want to accidentally expose environment secrets.

### Handlers

`src/handlers/` contains handler files for the bot. The `CommandHandler.js` file comes by default; it's purpose is to register commands statically so they can be minified, and to handle interaction (when a user uses a command).

**Best Practices**

- Don't create custom handlers unless you know what you're doing.
- As always, implement good error handling in the handler files you create.
- Use `.js` files only.

### Services

`src/services/` contains service files for the bot. It's purpose is to manage the bot's services, like Discord. [discord.js](src/services/discord.js) is responsible for registering the Discord client and logging in.
